## Notebook to pre-process activity data

### Description
The main objective of this notebook is to pre-process data for further aggregation and calculations.
In particular, it includes the following steps:
- Filter deleted or closed posts;
- Calculate number of activities for `tags`;
- Inherit `tags` field for answers from questions;
- Explode `tags` field so resulting data is flat for easier aggregations.

### Input 
This notebook takes as an input `posts.csv` file, produced by the previous step.

### Output
As an output this notebook produces `post_tag.csv` file with the following format:
```csv
Id,ActivityDate,ActivityScore,Tag
{post-id},{activity-date},{activity-score},{tag}
```
where:
- `{post-id}` - post identifier.
- `{activity-date}` - activity date in 'YYYY-MM' format. For example: '2008-07'
- `{activity-score}` - sum of all activities, such as votes, comments, post creation.
- `{tag}` - single tag related to a post. For instance: `c#`

For instance:
```csv
Id,ActivityDate,ActivityScore,Tag
``` 

In [11]:
import dask.dataframe as dd
from config import get_file_path

#### Load data, show shape and sample

In [12]:
posts_file_path = get_file_path("posts.csv")
all_posts_df = dd.read_csv(posts_file_path)

In [13]:
all_posts_df.dtypes

CreationDate     string[pyarrow]
Id                         int64
CommentCount               int64
ViewCount                float64
Score                      int64
PostTypeId                 int64
ClosedDate       string[pyarrow]
ParentId                 float64
Tags             string[pyarrow]
DeletionDate             float64
FavoriteCount            float64
dtype: object

In [14]:
all_posts_df

,CreationDate,Id,CommentCount,ViewCount,Score,PostTypeId,ClosedDate,ParentId,Tags,DeletionDate,FavoriteCount
npartitions=60,,,,,,,,,,,
,string,int64,int64,float64,int64,int64,string,float64,string,float64,float64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


### Filter data
Filter out closed or deleted posts, since they considered as irrelevant
'DeletionDate' and 'ClosedDate' columns present only for deleted or closed posts respectively.

In [15]:
non_deleted_closed_posts_df = all_posts_df[dd.isna(all_posts_df['DeletionDate']) & dd.isna(all_posts_df['ClosedDate'])]
non_deleted_closed_posts_count = len(non_deleted_closed_posts_df)
all_posts_count = len(all_posts_df)
print(all_posts_count)

non_deleted_closed_posts_percentage = round((non_deleted_closed_posts_count / all_posts_count) * 100, 2)
print(f'Number of NOT deleted or closed posts: {non_deleted_closed_posts_count}, which is {non_deleted_closed_posts_percentage}% of all data')

# Remove columns that we don't need anymore
posts_df = non_deleted_closed_posts_df.drop(['DeletionDate', 'ClosedDate'], axis=1)
print('Filtered posts dataframe shape:')
posts_df.dtypes

59749049
Number of NOT deleted or closed posts: 58652191, which is 98.16% of all data
Filtered posts dataframe shape:


CreationDate     string[pyarrow]
Id                         int64
CommentCount               int64
ViewCount                float64
Score                      int64
PostTypeId                 int64
ParentId                 float64
Tags             string[pyarrow]
FavoriteCount            float64
dtype: object

### Convert creation date
Trends will be calculated per-month granularity.
Convert creation datetime into year-month pair. It easier and faster to do here, while posts dataframe is relatively small.

In [19]:
posts_df['CreationDateNew'] = dd.to_datetime(posts_df['CreationDate']).dt.strftime('%Y-%m')
posts_df = posts_df.drop(['CreationDate'], axis=1).rename(columns={'CreationDateNew': 'CreationDate'})
posts_df.head()

,Id,CommentCount,ViewCount,Score,PostTypeId,ParentId,Tags,FavoriteCount,ActivityScore,CreationDate
0,4,4,76276.0,804,1,NaN,<c#><floating-point><type-conversion><double><...,0.0,809,2008-07
1,6,0,24428.0,320,1,NaN,<html><css><internet-explorer-7>,0.0,321,2008-07
2,7,0,NaN,528,2,4.0,NaN,NaN,529,2008-07
3,9,11,824304.0,2246,1,NaN,<c#><.net><datetime>,0.0,2258,2008-07
4,11,4,202471.0,1651,1,NaN,<c#><datetime><time><datediff><relative-time-s...,0.0,1656,2008-07


### Add activity score
Each event of any sort of activity (comments creationtion, up votes, post creation) is summed up in the following score.

In [20]:
posts_df['ActivityScore'] = posts_df['CommentCount'] + posts_df['Score'] + 1
posts_df.head()

,Id,CommentCount,ViewCount,Score,PostTypeId,ParentId,Tags,FavoriteCount,ActivityScore,CreationDate
0,4,4,76276.0,804,1,NaN,<c#><floating-point><type-conversion><double><...,0.0,809,2008-07
1,6,0,24428.0,320,1,NaN,<html><css><internet-explorer-7>,0.0,321,2008-07
2,7,0,NaN,528,2,4.0,NaN,NaN,529,2008-07
3,9,11,824304.0,2246,1,NaN,<c#><.net><datetime>,0.0,2258,2008-07
4,11,4,202471.0,1651,1,NaN,<c#><datetime><time><datediff><relative-time-s...,0.0,1656,2008-07


### Split posts into questions and answers

Split all posts_df onto two other dataframes: questions and answers.
Questions does not assigned tags and answers does.
Use `PostTypeId` column for it, where `1` is a type for question and `2` is for answer.
Drop `ParentId` column for questions, because it is always `null` since they are parent posts for questions.
Drop `Tags` column for answers, because it is always `null` since questions contain tags only. 
Answers should have same tags as questions.
See readme.txt for more details.

In [23]:
answers_df = posts_df[posts_df['PostTypeId'] == 2].drop(['PostTypeId', 'Tags'], axis=1)
questions_df = posts_df[posts_df['PostTypeId'] == 1].drop(['PostTypeId', 'ParentId'], axis=1)

In [25]:
answers_df.head()

,Id,CommentCount,ViewCount,Score,ParentId,FavoriteCount,ActivityScore,CreationDate
2,7,0,NaN,528,4.0,NaN,529,2008-07
5,12,10,NaN,348,11.0,NaN,359,2008-07
10,18,2,NaN,59,17.0,NaN,62,2008-08
12,22,1,NaN,56,9.0,NaN,58,2008-08
15,26,0,NaN,143,17.0,NaN,144,2008-08


### Parse tags

Parse `Tags` column. It contains list of tags in XML like format. For instance: `<c#><.net><datetime>`
To work properly with it, we need to turn it into proper list of tags.

In [31]:
# Remove the '<' and '>' characters and then split by '><'
questions_df['TagsParsed'] = questions_df['Tags'].str.replace('<', '').str.replace('>', '<').str.split('<')

# Remove 'Tags' column that is not needed anymore
questions_df = questions_df.drop(['Tags'], axis=1)

# Remove empty strings that may appear as a result of the split
questions_df['TagsParsed'] = questions_df['TagsParsed'].apply(lambda tags: [tag for tag in tags if tag], meta=('TagsParsed', 'object'))
questions_df.head()

,Id,CommentCount,ViewCount,Score,FavoriteCount,ActivityScore,CreationDate,TagsParsed
0,4,4,76276.0,804,0.0,809,2008-07,"[c#, floating-point, type-conversion, double, ..."
1,6,0,24428.0,320,0.0,321,2008-07,"[html, css, internet-explorer-7]"
3,9,11,824304.0,2246,0.0,2258,2008-07,"[c#, .net, datetime]"
4,11,4,202471.0,1651,0.0,1656,2008-07,"[c#, datetime, time, datediff, relative-time-s..."
6,13,10,304281.0,718,0.0,729,2008-08,"[html, browser, timezone, user-agent, timezone..."


### Explode tags

Explode 'TagsParsed' column to have a single tag per row and rename it to 'Tag'
Having single tag per row allows to perform necessary aggregations.

In [38]:
questions_tag_df = questions_df.explode('TagsParsed').rename(columns={'TagsParsed': 'Tag'})[['Id', 'ActivityScore', 'CreationDate', 'Tag']]
questions_tag_df.head()

,Id,ActivityScore,CreationDate,Tag
0,4,809,2008-07,c#
0,4,809,2008-07,floating-point
0,4,809,2008-07,type-conversion
0,4,809,2008-07,double
0,4,809,2008-07,decimal


### Assign tags on answers

Merge answers dataframe with questions dataframe on 'Id' and 'ParentId' columns. 
This merge is needed to populate tag data into answers posts for later aggregations.
As it was mentioned before, answers posts don't have tags assigned, because they implicitly inherit those from parent question posts.

In [39]:
answers_tag_df = dd.merge(
    questions_tag_df,
    answers_df,
    left_on='Id',
    right_on='ParentId',
    how='inner',
    suffixes=('_Question', '_Answer')
)[['ActivityScore_Answer', 'CreationDate_Answer', 'Tag', 'Id_Answer']]\
    .rename(columns={'ActivityScore_Answer': 'ActivityScore','CreationDate_Answer': 'CreationDate', 'Id_Answer': 'Id'})
answers_tag_df

/opt/conda/lib/python3.10/site-packages/dask/dataframe/multi.py:520: UserWarning: Merging dataframes with merge column data type mismatches: 
+--------------------+------------+-------------+
| Merge columns      | left dtype | right dtype |
+--------------------+------------+-------------+
| ('Id', 'ParentId') | int64      | float64     |
+--------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


,ActivityScore,CreationDate,Tag,Id
npartitions=60,,,,
,int64,object,object,int64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


#### Union questions and answers
Union questions and answers dataframes to get dataframe that shows all posts created for particular tag.

In [45]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

posts_tag_df = dd.concat([questions_tag_df, answers_tag_df], ignore_index=True)

### Save
Save final dataframe as intermediate result

In [46]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

posts_tag_file_path = get_file_path("posts_activity_tag.csv")
posts_tag_df.to_csv(posts_tag_file_path, index=False)

[########################################] | 100% Completed | 23m 30s
[########################################] | 100% Completed | 23m 31s
[########################################] | 100% Completed | 23m 31s
[########################################] | 100% Completed | 23m 31s
[########################################] | 100% Completed | 23m 31s


['/home/jupyter/data/analysis/posts_activity_tag.csv/000.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/001.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/002.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/003.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/004.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/005.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/006.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/007.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/008.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/009.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/010.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/011.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/012.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/013.part',
 '/home/jupyter/data/analysis/posts_activity_tag.csv/014.part',
 '/home/jupyter/data/analysis/posts_acti